Product Page Crawler For Carefour France 
Author Mohit Khandelwal

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
from selenium.webdriver.support import expected_conditions as EC

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)

C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\472493040.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)
C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\472493040.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)


In [3]:
inputlist = pd.read_excel(r"carefourinputlist missing.xlsx")
inputlist

,Asin
0,3000000040676
1,3182670104732
2,3663645077645
3,9782375543689
4,9782749171845
5,3165140755849
6,3221614004889
7,3276550232704
8,3414970566416
9,3664061908209


In [4]:
inputlist.drop_duplicates(inplace=True)
inputlist.dropna(inplace = True)
asinlist = list(inputlist['Asin '].unique())
asinlist

[3000000040676,
 3182670104732,
 3663645077645,
 9782375543689,
 9782749171845,
 3165140755849,
 3221614004889,
 3276550232704,
 3414970566416,
 3664061908209,
 4059952610573,
 4059952610610,
 4250559917670,
 5411478348016,
 7332543747153,
 3276550394761,
 3276552413026,
 3276552672836,
 3276552684280,
 3276554577290,
 3276558257211,
 3276558413600,
 3276558453453,
 3276558939209,
 3523680304352,
 3523680473669,
 3615616763194,
 3615616763200,
 3615616763217,
 3666373443577,
 3666373880631,
 3666472262246,
 3666472262253,
 4006825659061,
 4006825667189,
 8425490030818,
 8425490136084,
 8435527818340,
 8435574358745,
 8436538305812,
 8712184030309]

In [5]:
df = pd.DataFrame(columns=['Asin','Category','Category Tree','Product URL','Product Title','Product Description','Product Features',
                            'Average Rating','Review Count','Ingredients','Product Benefits','Conservation','Product Characterstics',
                           'Operator & Consumer Service','Consumer Benefits','Price','Price Per Quanitity',
                           'Quantity','Allergens','Images URL'])

In [7]:
for val in tqdm(range(0,len(asinlist))):
    
    i=asinlist[val]
    print(i)
    carefoururl = "https://www.carrefour.fr/p/-{}".format(i)
    driver.get(carefoururl)
    print(carefoururl)
    
    
    #click all the seemore available 
    #ingredient
    try:
        voir_plus_button = WebDriverWait(driver, 6).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[4]/div[1]/div/button")))
        voir_plus_button.click()
        time.sleep(2)
    except:
        pass
    #productbenefit
    try:    
        voir_plus_button = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div/div/button")))
        voir_plus_button.click()
        
    except:
        pass
    #consumerbenefit
    try:
        voir_plus_button = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div[2]/div/button")))
        voir_plus_button.click()
        
    except:
        pass
    
    #refreshing the page
    soup=BeautifulSoup(driver.page_source,"html.parser")
    productcomplete=soup.find_all("div",class_="pdp__product")
    # Initialize an empty DataFrame to store the extracted data
    try:
        aa=soup.find_all('li',class_="breadcrumb-trail__item")
        categorytree=[x.text.strip() for x in aa[:-1]]
        print(categorytree)
        
    except:
        categorytree=""
            
        
    #category
    try:
        cat1 = soup.find("div",class_="regulated-name")
        category=cat1.find("p",class_="pl-text pl-text--size-m pl-text--style-p").text
        print(category)
    except:
        category=""
    #Title
    try :
        prodtitle = soup.find('h1', class_='pl-text--style-subtitle').get_text(strip=True)
        print("Product Title:",prodtitle) 
    except:
        try:
            title2= soup.find("div",class_="main-details--non-food__header")
            finaltitle=title2.find("h1",class_="pdp-card__title pl-text pl-text--size-m pl-text--style-subtitle").text
            prodtitle=finaltitle
            print("Product Title:",prodtitle)
        except:
            prodtitle=""

    #description
    try:
        product_description = soup.find('div', class_='pl-text--style-p').get_text(strip=True)
    # Print the product description
        print("Product Description:",product_description)
    except:
        try:
            desc2=soup.find("div",class_="product-characteristics__description").text
            product_description=desc2
            print("Product Description:",product_description)
        except:
            product_description=""
        


    #nutritional values 
    try:
        nutritional_values_table = soup.find('table', class_='nutritional-details')
        nutritional_values_rows = nutritional_values_table.find_all('tr', class_='nutritional-fact')
        nutritional_values_dict = {}

        for row in nutritional_values_rows:
            name = row.find('th', class_='nutritional-fact__name').get_text(strip=True)
            value_per_quantity = row.find('td', class_='nutritional-fact__value nutritional-fact__center').get_text(strip=True)
            value_percent_element = row.find('td', class_='nutritional-fact__value nutritional-fact__right')
            value_percent = value_percent_element.get_text(strip=True) if value_percent_element else None

            nutritional_values_dict[name] = {
                "value_per_quantity": value_per_quantity,
                "value_percent": value_percent
            }

        # Print the fetched nutritional details
        print("Nutritional Values:")
        for name, values in nutritional_values_dict.items():
            if values["value_percent"]:
                print(f"{name}: {values['value_per_quantity']} ({values['value_percent']})")
            else:
                print(f"{name}: {values['value_per_quantity']}")
    except:
        nutritional_values_dict=""
   
         #product benefits 
    try:
#         voir_plus_button = WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div/div/button")))
#         voir_plus_button.click()
    
#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#         updated_html = driver.page_source
#         soup = BeautifulSoup(updated_html, "html.parser")

        # Find the "Ingrédients" section
        benefit= soup.find("div", class_="product-benefits secondary-details__block")
        productb= benefit.find_all("div",class_="product-block-content")
        productbenefit=productb[0].get_text(strip=True)
        # Print the "Ingrédients" text
        print(productbenefit)
    except:
        productbenefit=""
 
        #consumer benefits
    try:
#         voir_plus_button = WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div[2]/div/button")))
#         voir_plus_button.click()

#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#         updated_html1 = driver.page_source
#         soup1 = BeautifulSoup(updated_html1, "html.parser")

        # Find the "Ingrédients" section
        benefit= soup.find("div", class_="product-benefits secondary-details__block")
        consumerb= benefit.find_all("div",class_="product-block-content")
        consumerbenefit=consumerb[1].get_text(strip=True)
        # Print the "Ingrédients" text
        print(consumerbenefit)
    except:
        consumerbenefit=""

    #product allergens
    try:
        div_element = soup.find("div", {"data-testid": "product-composition-allergens", "class": "product-composition-allergens"})
        allergens = div_element.find("span", class_="pl-text pl-text--size-m pl-text--style-p").text
        print("Allergens:",allergens)
    except:
        allergens =""
    
    #price 
    finalprice = ""  # Initialize the finalprice variable as an empty string

    try:
        price_element = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
        price = price_element.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final"}).text
        finalprice = price  # If price is available, assign it to finalprice
    except AttributeError:  # Handle the AttributeError in case the price is not available
        try:
            price_element2 = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
            discountedprice = price_element2.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--promo"}).text
            finalprice = discountedprice  # If discounted price is available, assign it to finalprice
        except AttributeError:  # Handle the AttributeError in case the discounted price is also not available
            try:
                price_element3 = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
                discountedprice1 = price_element3.find("span", {"class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--fid"}).text
                finalprice = discountedprice1  # If discounted price is available, assign it to finalprice
            except AttributeError:
                try:
                    price3=soup.find("div",class_="product-card-price")
                    promo_price=price3.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--promo"}).text
                    finalprice=promo_price
                except:
                    finalprice = "" 
    print(finalprice)
    

    #price_per_quantity
    try:
        price_per = soup.find("div",class_="pdp-pricing__group")
        priceperquantity = price_per.find("p",class_="pl-text pl-text--size-s pl-text--style-p").text
    
    except:
        priceperquantity =""
    
    #quanitity
    try:
        quantity_element = soup.find("div",class_="main-details__header mobile-only")
        quantity = quantity_element.find("span",class_="pl-text pl-text--size-m pl-text--style-p pl-text--bold").text
    
    except:
        quantity =""
    
    #image data
    #first block is for single image
    try:
        image_element1 = soup.find("div", class_="single-image__wrapper")
        zoomable_image_div = image_element1.find("div", class_="zoomable-image")
        image_element = zoomable_image_div.find("img", class_="zoomable-image__image")
        image_url = image_element["data-src"]
        print(image_url)
    except: #second block is for multiple image
        try:
            image_elements = soup.find_all('img', class_="image ds-visual-picker__image")
            product_images = [img['src'] for img in image_elements]
            product_images1 = [x.replace("p_43x43", "p_540x540") for x in product_images]
            print("Product Images:")
            for image_url in product_images1:
                print(image_url)
        except:
            image_url = ""
            print("Product Image not available.")

    #rating&review Summary
    try:
         # Find the rating summary element and extract the text
        rating_summary_element = soup.find('p', class_='rating-summary__average')
        rating_summary = rating_summary_element.get_text(strip=True)

        # Find the review count element and extract the text
        review_count_element = soup.find('p', class_='rating-summary__count')
        review_count = review_count_element.get_text(strip=True)
        # Print the rating summary and review count
        print("Average Rating:", rating_summary)
        print("Review Count:", review_count)
    except:
        rating_summary=""
        review_count=""
        
    product_features = ""  # Define product_features as an empty string
    #Product Features
    try:
        product_features_heading = soup.find("div",class_="product-badge-anchor-heading").text
        content_features = soup.find("div",class_="product-badge-anchor-content").text
        # Format and print the output
        product_features = f"{product_features_heading} : {content_features}"
    
    except:
        try:
            # Find all the <li> tags with class "product-key-features__item"
            features_items = soup.find_all("li", class_="product-key-features__item")

            # Extract the text content from each <span> tag
            features = [item.span.get_text(strip=True) for item in features_items]

            # Print the extracted features
            for product_features in features:
                print("•",product_features)
        except:
            product_features = ""
                
     #product characterstics
    try:
        char_features=soup.find("div",class_="product-characteristics__description").text
        print("Product Characterstics",char_features)
    except:
        try:
            char_features=soup.find("div",class_="product-flags secondary-details__block").text
            print("Product Characterstics",char_features)
        except:
            char_features=""
    #Advice & Consumer Info
    try:
        advice_consumer_info = soup.find("div",class_="product-informations secondary-details__block")
        advice_consumer_info1 = advice_consumer_info.find_all("span",class_="pl-text pl-text--size-m pl-text--style-p")
        conservation=advice_consumer_info1[0].text
        Operator_contact_Consumer_service=advice_consumer_info1[1].text
        
    except:
        conservation=""
        Operator_contact_Consumer_service=""
        
      
    #Ingredient Complete
#     try:
#         while True:
#             voir_plus_button = WebDriverWait(driver,5).until(
#             EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[4]/div[1]/div/button")))
#             voir_plus_button.click()
            
#     except:
#         pass
#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#     updated_html = driver.page_source
#     soup1 = BeautifulSoup(updated_html, "html.parser")
    
    try:
        # Find the "Ingrédients" section
        ingredient1= soup.find("div", {"data-testid": "product-composition-default-content", "class": "product-block-content"})
        ingredient= ingredient1.find("span", class_="pl-text pl-text--size-m pl-text--style-p").text
        # Print the "Ingrédients" text
        print(ingredient)
    except:
        ingredient=""
    
    # Append the extracted information as a new row to the DataFrame
    data = {'Asin':i,'Product URL': carefoururl,'Category': category,'Category Tree': categorytree,
            'Nutritional Value':nutritional_values_dict,
            'Product Title': prodtitle,
            'Product Description': product_description,
            'Average Rating': rating_summary,
            'Review Count': review_count,
            'Ingredients': ingredient,
            'Product Features': product_features,
            'Product Benefits': productbenefit,
            'Consumer Benefits': consumerbenefit,
            'Product Characterstics':char_features,
            'Price': finalprice,
            'Price Per Quanitity': priceperquantity,
            'Quantity': quantity,
            'Allergens': allergens,
            'Images URL': image_url,
            'Conservation': conservation,'Operator & Consumer Service': Operator_contact_Consumer_service}
    df = df.append(data, ignore_index=True)
    time.sleep(1)

# Print the extracted shop details
print(df)

  0%|          | 0/41 [00:00<?, ?it/s]

3000000040676
https://www.carrefour.fr/p/-3000000040676
['Accueil', 'Rayons', 'Fruits et Légumes', 'Légumes', 'Salades et Tomates', 'Tomates']
Product Title: Tomates rondes charnues à farcir vrac
Product Description: En ce moment
 2,19€ 
https://media.carrefour.fr/medias/245c99ada1ad30c08b531be6423048a5/p_540x540/3000000040676-photosite-20170627-162038-0.jpg
Product Characterstics 
Tomates rondes charnues


C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\1533287878.py:320: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


3182670104732
https://www.carrefour.fr/p/-3182670104732
['Accueil', 'Rayons', "Jardin et Aménagement d'extérieur", 'Potager', 'Graines']
Product Title: Sachet Graines Les Tropicales - Tomate Heinz 1370
Product Description: En ce moment

https://media.carrefour.fr/medias/b392e6c27ab73c68ae19d062057bfd1e/p_540x540/750265bd2a1d40f29573f903a9368525-image.webp
Product Characterstics Variété à croissance déterminée, se cultive en saisons fraîche et sèche sans tuteur. Ses fruits ronds, très fermes ont une remarquable résistance à l'éclatement. Production groupée. Culture recommandée en zone tropicale pour sa résistance au fusarium


C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\1533287878.py:320: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


...

8436538305812
https://www.carrefour.fr/p/-8436538305812
['Accueil', 'Rayons', 'Electroménager', 'Entretien du sol', 'Aspirateurs à main']
Product Title: Aspirateur À Main Sans Fil Rechargeable - Rouge. Vert Et Bleu Assortis
Product Description: En ce moment
 39,90€ 
Product Images:
https://media.carrefour.fr/medias/2f5a0857e30f3cdeb56ee0ce4edea543/p_540x540/58f1fcf4f8b14ead814b2909cb05e188-image.jpg
https://media.carrefour.fr/medias/cf9fc5c1c4fb37faa5312a1775e123d8/p_540x540/d0a4a5ef04314b7d8e80ee6484327573-image.jpg
https://media.carrefour.fr/medias/912f35cf3b9435549a58d278da4661bc/p_540x540/364d8061969f439bb9820e3af305a859-image.jpg
https://media.carrefour.fr/medias/b23c29acc3f43cf19526c629b1f3482c/p_540x540/e1be07e0f7ef43f58adf921ff1a7823a-image.jpg
Product Characterstics Aspirateur à main rechargeable
Grand pouvoir d'aspiration: 2,1 kPa
Pour les surfaces sèches et humides
Alimentation: batterie Ni-MH 7.2 V
Base de chargement mural
Filtre permanent et lavable
Accessoires: pinceau d'

C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\1533287878.py:320: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


8712184030309
https://www.carrefour.fr/p/-8712184030309
['Accueil', 'Rayons', 'Electroménager', 'Entretien du sol', 'Aspirateurs traineau']
Product Title: Aspirateur Sans Fil À Main Avc225w Bestron
Product Description: En ce moment
 45,30€ 
Product Images:
https://media.carrefour.fr/medias/5e1f74da3cf632c38aff2831a57403e6/p_540x540/02d6cda3cebb482793f2fd9c38e05773-image.jpg
https://media.carrefour.fr/medias/9f54e0a709e634248ddc21f9b06d564a/p_540x540/79e7f37ef38c4fa39ec5458a849ef9c7-image.jpg
https://media.carrefour.fr/medias/e702e39016a534f8affb92ae1ec6d588/p_540x540/dd9c596bfd714f1091953acb0519ff32-image.jpg
https://media.carrefour.fr/medias/e4199b14f27e3adaa0b1d8fd4df147c4/p_540x540/0f317f66abe341be9aeccfab87530cc0-image.jpg
Product Characterstics L'aspirateur Bestron AVC225W gère à la fois la poussière et les liquides. Il dispose d'une batterie puissante et écologique, qui fonctionne pendant près de 10 minutes. Le filtre est lavable. Il est équipé d'un embout en caoutchouc pour les 

C:\Users\mohit\AppData\Local\Temp\ipykernel_5972\1533287878.py:320: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


             Asin              Category  \
0   3000000040676                         
1   3182670104732                         
2   3663645077645                         
3   9782375543689                         
4   9782749171845                         
5   3165140755849                         
6   3221614004889                         
7   3276550232704                         
8   3414970566416                         
9   3664061908209                         
10  4059952610573                         
11  4059952610610                         
12  4250559917670                         
13  5411478348016                         
14  7332543747153                         
15  3276550394761                         
16  3276552413026                         
17  3276552672836                         
18  3276552684280                         
19  3276554577290                         
20  3276558257211                         
21  3276558413600         Avocat SIMPL    
22  3276558

In [8]:
df.head()

,Asin,Category,Category Tree,Product URL,Product Title,Product Description,Product Features,Average Rating,Review Count,Ingredients,...,Conservation,Product Characterstics,Operator & Consumer Service,Consumer Benefits,Price,Price Per Quanitity,Quantity,Allergens,Images URL,Nutritional Value
0,3000000040676,,"[Accueil, Rayons, Fruits et Légumes, Légumes, ...",https://www.carrefour.fr/p/-3000000040676,Tomates rondes charnues à farcir vrac,En ce moment,,,,Tomates rondes charnues,...,,,,,"2,19€",2.19 € / KG,le kilo,,https://media.carrefour.fr/medias/245c99ada1ad...,
1,3182670104732,,"[Accueil, Rayons, Jardin et Aménagement d'exté...",https://www.carrefour.fr/p/-3182670104732,Sachet Graines Les Tropicales - Tomate Heinz 1370,En ce moment,,,,,...,,"Variété à croissance déterminée, se cultive en...",,,,,,,https://media.carrefour.fr/medias/b392e6c27ab7...,
2,3663645077645,,"[Accueil, Rayons, Jeux et Jouets, Jeu d'imitat...",https://www.carrefour.fr/p/-3663645077645,Poele Couvercle Glace Hamburger Ketchup Jouet ...,En ce moment,,,,,...,,"Très amusant, ce set comprends : 1 poele ( 18 ...",,,,,,,https://media.carrefour.fr/medias/5a3e0ac638a3...,
3,9782375543689,,"[Accueil, Rayons, Livres et Culture, Littératu...",https://www.carrefour.fr/p/-9782375543689,"Amour, Cuisine Et Poulet Sauce Moutarde",En ce moment,,,,,...,,"Juliette a la cuisine dans le sang, elle veut ...",,,,,,,https://media.carrefour.fr/medias/1874497397d1...,
4,9782749171845,,"[Accueil, Rayons, Livres et Culture, Livre vie...",https://www.carrefour.fr/p/-9782749171845,L'oeuf Mayo,En ce moment,,,,,...,,Consacrer un livre entier à une préparation au...,,,,,,,https://media.carrefour.fr/medias/9bdfaa7f79e2...,


In [9]:
df.to_excel(r'finaloutput_carefour_remaining40.xlsx')

In [ ]:
soup=BeautifulSoup(driver.page_source,"html.parser")
productcomplete=soup.find_all("div",class_="pdp__product")

In [ ]:
soup=BeautifulSoup(driver.page_source,"html.parser")
productcomplete=soup.find_all("div",class_="pdp__product")

In [ ]:
price3=soup.find("div",class_="product-card-price")
promo_price=price3.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--promo"}).text

In [ ]:
promo_price

In [ ]:
# Find all the <li> tags with class "product-key-features__item"
features_items = soup.find_all("li", class_="product-key-features__item")

# Extract the text content from each <span> tag
features = [item.span.get_text(strip=True) for item in features_items]

# Print the extracted features
for feature in features:
    print("•",feature)

In [ ]:
soup=BeautifulSoup(driver.page_source,"html.parser")
productcomplete=soup.find_all("div",class_="pdp__product")

In [ ]:
#first block is for single image
try:
    image_element1 = soup.find("div", class_="single-image__wrapper")
    zoomable_image_div = image_element1.find("div", class_="zoomable-image")
    image_element = zoomable_image_div.find("img", class_="zoomable-image__image")
    image_url = image_element["data-src"]
    print(image_url)
except: #second block is for multiple image
    try:
        image_elements = soup.find_all('img', class_="image ds-visual-picker__image")
        product_images = [img['src'] for img in image_elements]
        product_images1 = [x.replace("p_43x43", "p_540x540") for x in product_images]
        print("Product Images:")
        for image_url in product_images1:
            print(image_url)
    except:
        image_url = ""
        print("Product Image not available.")

In [ ]:
feature3=soup.find("div",class_="product-flags secondary-details__block").text

In [ ]:
try:
        product_features_heading = soup.find("div",class_="product-badge-anchor-heading").text
        content_features = soup.find("div",class_="product-badge-anchor-content").text
        # Format and print the output
        product_features = f"{product_features_heading} : {content_features}"
    
except:
    try:
        # Find all the <li> tags with class "product-key-features__item"
        features_items = soup.find_all("li", class_="product-key-features__item")

        # Extract the text content from each <span> tag
        features = [item.span.get_text(strip=True) for item in features_items]

        # Print the extracted features
        for product_features in features:
            print("•",product_features)
    except:
        try:
            feature3=soup.find("div",class_="product-characteristics__description").text
            product_features=feature3
        except:
            product_features = ""

In [ ]:
feature3=soup.find("div",class_="product-characteristics__description").text

In [ ]:
feature3

In [ ]:
char_features=soup.find("div",class_="product-flags secondary-details__block").text